In [9]:
import xml.etree.ElementTree as et
from xml.etree.ElementTree import Element
import unicodedata
import os

In [10]:
# FIXME: names have to be proofread
NAME_TO_MSTRKS = [
  ({"Rast", "Evc", "Rehavi", "Pencgah", "Sazkar", "Sazkar", "Neva", "Neva Buselik", "Tahir", "Tahir-Buselik", "Huseyni", "Huseyni Asiran", "Gulizar", "Yegah", "Beyati Araban", "Muhayyer", "Muhayyer Buselik", "Gerdaniye",  "Gerdaniye Buselik", "Dilkes Haveran", "Evc-Buselik", "Irak", "Pesendide"}, ["B -0.5", "F 1"]), 
  ({"Acem Kurdi", "Acem Asiran", "Kurdi", "Muhayyer-Kurdi"}, ["B -1"]), 
  ({"Muhayyer Sunbule", "Saba", "Saba Buselik", "Saba-Zemzeme", 
  "Bestenigar", "Sevk Efza", "Sevk-i-Tarab", "Sevk u Tarab"}, ["B -0.5", "D -0.25"]), 
  ({"Ussak", "Beyati", "Beyati-Buselik", "Isfahan", "Berte-Isfahan", "Hisar"}, ["B -0.5"]), 
  ({"Nisaburek", "Evc Ara", "Ferahnak"}, ["F 1", "C 1"]),
  ({"Suzidilara", "Mahur", "Mahur-Buselik", "Zavil"}, ["F 0.25"]),
  ({"Nikriz", "Sehnaz"}, ["B -0.25", "C 1"]), 
  ({"Hisar-Buselik", "Suzdil"}, ["F 0.5", "G 1", "D 1"]), 
  ({"Segah", "Mustear"}, ["B -0.5","E -0.5", "F 1"]), 
  ({"Huzzam", "Karcigar", "Beyati Araban-Buselik"}, ["B -0.5", "E -0.25", "F 1"]), 
  ({"Kurdili Hicazkar"}, ["B -1", "E -1", "A -1"]), 
  ({"Hicazkar"}, ["B -0.5", "E -0.25", "A -0.25", "F 1"]), 
  ({"Neveser"}, ["B -0.25", "E -0.25", "F 1", "C 1"]), 
  ({"Hicaz", "Hicaz Asiran", "Hicaz-Buselik"}, ["B -0.25", "F 1", "C 1"]), 
  ({"Arazbar", "Arazbar-Buselik"}, ["B -0.5", "E -0.5"]), 
  ({"Buselik-Asiran"}, ["F 1"]) 
]

ALL_NAMES = set()
for names, _ in NAME_TO_MSTRKS:
  ALL_NAMES |= names

def mstrks(name):
  for names, strks in NAME_TO_MSTRKS:
    if name in names:
      return strks
  assert False, "couldn't find name"

NUM_TO_ACC = {
  "-1": "flat",
  "-0.5": "quarter-flat",
  "-0.25": "slash-flat",
  "0.25": "slash-quarter-sharp",
  "0.5": "quarter-sharp",
  "1": "sharp",
}

def strks_to_xml(strks):
  """

  Helper. Takes strks, which is a string for key signature in the format:
  "note Ahalf-steps"

  and turns them into a tuple of XML Elements.
  """

  key_step = strks[0] #just the first letter. varname \approx strname
  key_alter = strks[2:] #second til end is the number.
  key_accidental = NUM_TO_ACC[strks[2:]] #gives the right name

  step_element = Element("key-step")
  step_element.text = key_step
  alter_element = Element("key-alter")
  alter_element.text = key_alter
  accidental_element = Element("key-accidental")
  accidental_element.text = key_accidental

  return (
    step_element,
    alter_element,
    accidental_element
  )

def insertks(filepath, elements, outpath):
  """insert the key elements into the key node of the file

  :param filepath: file to use
  :param elements: key elements from strks_to_xml
  """
  tree = et.parse(filepath)
  root = tree.getroot()

  key_element = root.findall("./part[@id='P1']/measure[@number='1']/attributes/key")[0]
  key_element.clear()
  for element in elements:
    key_element.append(element)
  tree.write(outpath)

In [11]:
# https://stackoverflow.com/a/517974
def remove_accents(input_str):
  nfkd_form = unicodedata.normalize('NFKD', input_str)
  return u"".join([c for c in nfkd_form if not unicodedata.combining(c)]).replace("ı", "i")



In [12]:
folder_path = '../Corrected Transcriptions/MusicXML/'
out_path = '../With Key Signatures/'

In [13]:
for musicfile in os.listdir(folder_path): #"." refers to current directory.
  #now, we know the string of folder, and can access all the files within!
  if musicfile[0] == "." or not musicfile.endswith('.musicxml'):
    """
    Quick tech note: "." indicates a file is HIDDEN.
    We don't wish to access hidden files, like .DS_Store and .mscbackup.
    Therefore, if a file starts with ., we are guaranteed that it's hidden.
    Of course, we don't have pathological file names that intentionally
    start with ".", so we assume start with "." -> don't consider it.
    """
    print("doesn't end with .musicxml:", musicfile)
    print()
    continue
  #now we are faced with JUST files. Now modify!

  # modify musicfile to update the key signature
  maqam_name = None
  for name in ALL_NAMES:
    normalized_filepath = remove_accents(musicfile)
    if name in normalized_filepath and (
      maqam_name is None or len(name) > len(maqam_name)
    ):
      maqam_name = name

  # maqam must be in file name
  if maqam_name is None:
    print("couldn't find maqam for:", normalized_filepath)
    print()
    continue

  mstrks_cor = mstrks(maqam_name) #find the correct list of key sigs to modify

  key_elements = []
  for strks in mstrks_cor:
    key_elements += strks_to_xml(strks)

  #at this point mstrks must exist.
  insertks(
    folder_path + musicfile,
    key_elements,
    out_path + musicfile,
  ) #gives us correct modification

Pencgah False
Pencgah False
Pencgah False
couldn't find maqam for: CT 282 -284 Rahatu’l‐ervah Pesrevi [Cifte Duyek].musicxml

Pencgah False
couldn't find maqam for: CT 94 Nisabur Sema’i Hindularin.musicxml

Pencgah False
Pencgah False
Pencgah False
Pencgah False
Pencgah False
couldn't find maqam for: CT 125-126  Kocek Pesrevi.musicxml

Pencgah False
Pencgah False
doesn't end with .musicxml: .DS_Store

Pencgah False
Pencgah False
Pencgah False
Pencgah False
Pencgah False
couldn't find maqam for: CT 30 Kemani Tatyos Efendi’nin Suzinak Saz Sema’isi.musicxml

Pencgah False
couldn't find maqam for: CT 260 Maye Saz Sema’isi. ‘Osman Bey merhumun.musicxml

Pencgah False
Pencgah False
couldn't find maqam for: CT 35 Sultan Selim Han-i Salis Hazretleri'nin Buzurg Saz Sema'isi.musicxml

Pencgah False
Pencgah False
Pencgah False
Pencgah False
couldn't find maqam for: CT 333 Sevk Aver Saz Sema’isi.musicxml

Pencgah False
Pencgah False
Pencgah False
Pencgah False
Pencgah False
Pencgah False
Pencgah F